# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
from pyspark.sql import functions as F

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included?   
[Kaggle Berkeley Earth Climate Change: Earth Surface Temperature Data
Exploring global temperatures since 1750](https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data)   
Source: [GlobalLandTemperaturesByCity.csv](https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data?select=GlobalLandTemperaturesByCity.csv)
- dt (object)  
- AverageTemperature (float64)  
- AverageTemperatureUncertainty (float64)  
- City (object)  
- Country (object)  
- Latitude (object)  
- Longitude (object)

In [2]:
# Read in the data here
fname = '../../data2/GlobalLandTemperaturesByCity.csv'
df = pd.read_csv(fname)

In [3]:
df.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8599212 entries, 0 to 8599211
Data columns (total 7 columns):
dt                               object
AverageTemperature               float64
AverageTemperatureUncertainty    float64
City                             object
Country                          object
Latitude                         object
Longitude                        object
dtypes: float64(2), object(5)
memory usage: 459.2+ MB


In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.\
config("spark.jars.repositories", "https://repos.spark-packages.org/").\
config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11").\
enableHiveSupport().getOrCreate()

df_spark = spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')


In [ ]:
#write to parquet
df_spark.write.parquet("sas_data")
df_spark=spark.read.parquet("sas_data")

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.
- The data contain a lot NaN value   

#### Cleaning Steps
Document steps necessary to clean the data
- perform dropna() to clean the data

In [6]:
# Performing cleaning tasks here
df.dropna(inplace=True)
df['year']=df['dt'].apply(lambda x: x[:4])
df['month']=df['dt'].apply(lambda x: x[5:7])
df.drop('dt',axis=1,inplace=True)
df=df[['year','month','AverageTemperature','City','Country','Latitude','Longitude']]
df['Latitude']=df['Latitude'].str.strip('N')
df['Longitude']=df['Longitude'].str.strip('E')
df.head()

,year,month,AverageTemperature,City,Country,Latitude,Longitude
0,1743,11,6.068,Århus,Denmark,57.05,10.33
5,1744,04,5.788,Århus,Denmark,57.05,10.33
6,1744,05,10.644,Århus,Denmark,57.05,10.33
7,1744,06,14.051,Århus,Denmark,57.05,10.33
8,1744,07,16.082,Århus,Denmark,57.05,10.33


In [7]:
df.describe()

,AverageTemperature
count,8.235082e+06
mean,1.672743e+01
std,1.035344e+01
min,-4.270400e+01
25%,1.029900e+01
50%,1.883100e+01
75%,2.521000e+01
max,3.965100e+01


### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks  
- Checks if the dataframe is not empty.
- Checks if the given keys are a unique key in the dataframe.
- Checks if the dataframe has the expected dtypes.
- Checks if the dataframe has the expected ammount of columns.

In [ ]:
# Perform quality checks here
columns_num = 
keys = 
expect_dtype = 

if df.shape()[0] > 0:
    print("The dataframe is not empty.")
else:
    raise ValueError("DataFrame is empty!")
        
if df.select(F.countDistinct(*keys)).first()[0] == df.shape()[0]:
    print("The given keys are a unique key in the dataframe.")
else:
    raise ValueError("The given keys are not unique key!")
 
 if df.dtypes == expect_dtype:
    print("The dataframe has the expected dtypes.")
else:
    raise ValueError("The dataframe do not match with expected dtypes!")    
    
if df.shape()[1] == columns_num:
    print("The dataframe has the expected ammount of columns.")
else:
    raise ValueError("The dataframe does not has the expected ammount of columns!")

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.